# Java Running with Remote Debugging
Detects a JAVA process running with remote debugging allowing more than just localhost to connect

## Rule Content
```
- title: Java Running with Remote Debugging
  id: 8f88e3f6-2a49-48f5-a5c4-2f7eedf78710
  description: Detects a JAVA process running with remote debugging allowing more
    than just localhost to connect
  author: Florian Roth
  tags:
  - attack.discovery
  - attack.t1046
  logsource:
    category: process_creation
    product: windows
    service: null
  detection:
    selection:
      CommandLine: '*transport=dt_socket,address=*'
    exclusion:
    - CommandLine: '*address=127.0.0.1*'
    - CommandLine: '*address=localhost*'
    condition: selection and not exclusion
  fields:
  - CommandLine
  - ParentCommandLine
  falsepositives:
  - unknown
  level: medium

```

## Querying Elasticsearch

### Import Libraries

In [ ]:
from elasticsearch import Elasticsearch
from elasticsearch_dsl import Search
import pandas as pd

### Initialize Elasticsearch client

In [ ]:
es = Elasticsearch(['http://helk-elasticsearch:9200'])
searchContext = Search(using=es, index='logs-*', doc_type='doc')

### Run Elasticsearch Query

In [ ]:
s = searchContext.query('query_string', query='(process_command_line.keyword:*transport\=dt_socket,address\=* AND (NOT (process_command_line.keyword:*address\=127.0.0.1* OR process_command_line.keyword:*address\=localhost*)))')
response = s.execute()
if response.success():
    df = pd.DataFrame((d.to_dict() for d in s.scan()))

### Show Results

In [ ]:
df.head()